## ASSUMPTION
This example is written assuming we have a service up and running for us to utilise

Copyright IBM Corp. 2022, 2023
@author James Strudwick IBM Research

## Dependencies

In [ ]:
#For boc
from boc import BOC
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#For loading data
import pandas as pd
import numpy as np
# from io import StringIO

## Utility function

In [3]:
# np.maximum.accumulate([1,2,1,2])

In [4]:
def plot_progress(boc):
    exp_details = boc.get_experiment_details()['submittedVals']
    domain, targets = exp_details['X'], exp_details['y']

    plt.figure(figsize=(10,10))
    plt.plot(np.maximum.accumulate(targets), marker='o', linestyle='--', label='current best')
    plt.plot(targets, marker='*', linestyle='', color='r', label='epoch objective value')
#     plt.axhline(-1.0316, linestyle='--')
    plt.legend()
        
    targets = np.array(targets)
    X = np.array(domain)
    
    best_ind = np.argmax(targets)
    best_X = X[best_ind]
    best_y = targets[best_ind]
    
    plt.plot(np.maximum.accumulate(targets), marker='o', linestyle='--', color='b', label='current best')
    plt.plot(targets, marker='*', linestyle='', color='r', label='objective')
    plt.title(f'Best X found: {best_X}, best objectives found: {best_y}')
    plt.show()

## Data loading

In [5]:
exp_data=pd.read_csv("experiment_list_sterimol_final_iteration1_derivatives.csv")

In [6]:
exp_data

,MCP dim1,MCP dim2,Catalyst dim1,Catalyst dim2,Solvent dim1,Solvent dim2,Cosolvent dim1,Cosolvent dim2,Additives dim1,Additives dim2,cosolvent vs solvent ratio,temp in degree celsius,cat. loading mol%,time in min,yield
0,-26.381313,1.414474,1.216737,0.535949,-11.231260,-9.071500,14.314211,-1.634429,0.0,0.0,0.50,21,10.0,2.0,58
1,-26.381313,1.414474,1.216737,0.535949,-11.231260,-9.071500,14.314211,-1.634429,0.0,0.0,0.50,21,10.0,4.0,61
2,-26.381313,1.414474,1.216737,0.535949,-11.231260,-9.071500,14.314211,-1.634429,0.0,0.0,0.50,21,10.0,8.0,60
3,-26.381313,1.414474,1.216737,0.535949,-11.231260,-9.071500,0.000000,0.000000,0.0,0.0,0.00,21,10.0,2.0,0
4,-26.381313,1.414474,1.216737,0.535949,-11.231260,-9.071500,0.000000,0.000000,0.0,0.0,0.00,21,10.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,-26.381313,1.414474,2.618682,-13.172341,10.244377,7.708617,14.314211,-1.634429,0.0,0.0,3.33,21,10.0,2.0,0
172,-26.381313,1.414474,1.216737,0.535949,-6.429210,5.053837,14.314211,-1.634429,0.0,0.0,1.43,21,2.0,2.0,0
173,-15.978039,-5.349408,1.216737,0.535949,-11.231260,-9.071500,14.314211,-1.634429,0.0,0.0,0.50,21,10.0,2.0,43
174,-13.596670,-5.310672,1.216737,0.535949,-11.231260,-9.071500,14.314211,-1.634429,0.0,0.0,0.50,21,10.0,2.0,40


## Define the objective Function

In [7]:
# results=[]
def exp_1(X):
#     print(X)
    outputs=[]   
    dat=exp_data.to_numpy()
    
    for imp in X:
        print(imp,type(imp))
        for line in dat:
            print(line,type(line))
            if np.isclose(imp.astype(np.float64),line[:-1].astype(np.float64)).all():
                outputs.append([line[-1]])
#     results2.append(outputs)
    return np.array(outputs)

## Set up an experiment configuration

In [14]:
#surrogate
dim_types = ['continuous',#MCP dim1 
             'continuous',#MCP dim2
             'continuous',#Catalyst dim1 
             'continuous',#Catalyst dim2
             'continuous',#solvent dim1
             'continuous',#solvent dim2
             'continuous',#cosolvent dim1
             'continuous',#cosolvent dim2
             'continuous',#cosolvent vs solvent ratio
             'continuous',#additives dim1
             'continuous',#additives dim2
             'continuous',#temp in degree celsius
             'continuous',#cat. loading mol%
             'continuous',#time in hrs
            ]
domain=pd.read_csv("domain_derivatives_2hrs_rtm_0additives_0.5ratio_2nd_half.csv").to_numpy().tolist()
acq_type = 'Thompson'
acq_kwargs = {'otypes': ['max'],  # list defining the objective types for each objective
              'normalise_x': True, # choice to normalise the domain values used by the model
              'normalise_y': True, # choice to normalise the target values used by the model
#               'epsilon': 0.1, # explore-exploit tradeoff
              }

opt_types = {'continuous': 'Pool'} # set internal optimiser to  use initial 
                                                    # random sampling followed by cobyla

# parameters for the the Pool and cobyla optimiser
opt_kwargs = {'continuous':
                   {'domain':domain, 'dim_types': dim_types}
               }
# choice to use sequential single sampling 
sampler_type = 'Basic'
sampler_kwargs = {'nsample': 1}

# initial selection of 1 random sample and vary the random seed from 1 to 5 for repeatability
bo_kwargs = {'init_obs': 1,
             'log_level': 2,
             'seed': 2}

# model specification of two scikit-learn GPs for the two objectives
model_type = 'scikit-learn'
# selection of an ARD Matern kernel with nu 1.5 and initial lengthscale values of 0.1 for the
# models of each objective
model_kwargs = {'kernel': {'kernel_type' : 'Matern', 
                           'kernel_kwargs' : {}}}

# final config
config = {'model_type': model_type,
          'model_kwargs': model_kwargs,
          'acq_type': acq_type,
          'acq_kwargs': acq_kwargs,
          'dict_opt_types': opt_types,
          'dict_opt_kwargs': opt_kwargs,
          'sampler_type': sampler_type,
          'sampler_kwargs': sampler_kwargs,
          'bo_kwargs': bo_kwargs,
          }

## Instantiate the client

When setting `IP` you **do not** need to include `http://` or `https://` or the port number. The port number is set via `PORT`

In [15]:
# instantiate the BOC client
TOKEN = "00000000-0000-0000-0005-f987fc055425"
# TOKEN = "00000000-0000-0000-0005-f987ffa41104"
IP = "caplonsgprd-1.securegateway.appdomain.cloud"
PORT = "16551"
boc = BOC(user_token=TOKEN, ip=IP, port=PORT)
boc.validate_connection()

'Connection live.  boss version  -  2.0.2'

As the platform develops you might find that you get a warning similar to the one below if so we recomend that you up/down grade to a compatibile version of BOC

In [16]:
correct_ver = boc.__version__
boc.__version__='2.0.0'
boc.validate_connection()
correct_ver=boc.__version__

In [17]:
print(correct_ver,boc.__version__,boc.validate_connection())

2.0.0 2.0.0 Connection live.  boss version  -  2.0.2


## Submit an experiment

If you wish to wait until the experiement is ready for use, then set `wait` to `True`. If you wish to submit and not bother waiting, set it to `False`

In [18]:
# create the experiment
boc.experiment_id = boc.generate_experiment(config, wait=True)

Non-200 response returned. Please check returned json.


Store the experiment ID to be used later on

In [12]:
# print(boc.get_experiment_logs())

Non 200 response. Response object returned
<Response [401]>


In [23]:
# boc.save_experiment_id('trial.yml')

To load previous experiment back in:

In [24]:
# boc.load_experiment_id('trial.yml')

## Experiment information

If you have multiple experiments or if you lose track of ids you can get a list of them by running:

In [19]:
boc.list_experiments(status=True)

Multiple Ids retrieved.


[['00000000-0000-0000-0006-01e1f7cbaf8d', 'Archived'],
 ['00000000-0000-0000-0006-01e38442f190', 'Archived'],
 ['00000000-0000-0000-0006-01eca7279d71', 'Archived'],
 ['00000000-0000-0000-0006-01f2e25bbe24', 'Ready']]

To restore experiment

If you want to know the status of your experiments as well then you can set `status` to `True`. If not you can set it to `False` or leave it out.

Then if you wish to switch to one of them you can set that by running:

In [15]:
# boc.experiment_id = '00000000-0000-0000-0006-0059b78057f6'

## Run BOA sequentially

In [28]:
#N.A

To See the experiment records and actual errors:


In [22]:
# plot_progress(boc)

## Or interactively

In [23]:
# condition=exp_data.to_numpy()[:,:-1]
# len(condition)

173

In [33]:
yields=exp_data.to_numpy()[:,-1]
# len(yields_int)

In [34]:
# Return evaluated target values to BOA
condition=exp_data.to_numpy()[:,:-1]
yields_int=np.array([int(i) for i in yields]).reshape(-1,1)
boc.update_experiment(condition, yields_int)

'bayesopt updated'

In [36]:
# get an input batch
X_list=[]
for i in range(6):
    raw_X = boc.get_next_X()
    X_list.append(raw_X)
suggestions=np.array(X_list).reshape(6,14)
suggestions_data=pd.DataFrame(suggestions,columns=[
                     'MCP dim1', 
                     'MCP dim2',
                     'Catalyst dim1', 
                     'Catalyst dim2', 
                     'Solvent dim1', 
                     'Solvent dim2', 
                     'Cosolvent dim1', 
                     'Cosolvent dim2', 
                     'additive dim1',
                     'additive dim2',
                     'cosolvent vs solvent ratio',
                     'temp in degree celsius', 
                     'cat. loading mol%',
                     'time in hours']).to_csv('suggestions_data_derivatives1.csv')

In [13]:
# print(boc.get_experiment_logs())

Non 200 response. Response object returned
<Response [401]>


## Decode

In [37]:
MCP_SMILES=[r'C1(/C=C2CC\2)=CC=CC=C1', 
r'BrC(C=C1)=CC=C1/C=C2CC\2', 
r'ClC(C=C1)=CC=C1/C=C2CC\2', 
r'FC(C=C1)=CC=C1/C=C2CC\2', 
r'O=[N+](C(C=C1)=CC=C1/C=C2CC\2)[O-]',
r'COC(C=C1)=CC=C1/C=C2CC\2',
r'CN(C(C=C1)=CC=C1/C=C2CC\2)C',
r'CC(C)C(C=C1)=CC=C1/C=C2CC\2',
r'C1(/C=C2CC\2)=CC=C(C3=CC=CC=C3)C=C1',
r'CC1=CC(/C=C2CC\2)=CC=C1',
r'BrC1=CC(/C=C2CC\2)=CC=C1',
r'CC(C)C1=CC(/C=C2CC\2)=CC=C1',
r'FC(C(C=C1)=CC=C1/C=C2CC\2)(F)F',
r'CC(C=C1)=CC=C1/C=C2CC\2']#2-CH3'CC1=CC=CC=C1/C=C2CC\2' deleted due to no Hammett Const
Catalyst_SMILES=['CC1=CC=C(S(O)(=O)=O)C=C1',
'O=S(C1=CC=CC=C1)(O)=O',
'CS(=O)(O)=O',
'[H]Cl',
'F[B-](F)(F)F.[H+]',
'CC1(C)[C@]2([H])CC[C@@]1(CS(O)(=O)=O)C(C2)=O',
'OC1=C(F)C(F)=C(F)C(F)=C1F','O=S(O)(C(F)(F)F)=O',
'[H]Br','F[P-](F)(F)(F)(F)F.[H+]',
'O=C(O)C(F)(F)F',
'O=P(O)(O)O',
'O=Cl(=O)(O)=O',
'[H]I',
'O=P(OC1=CC=CC=C1)(OC2=CC=CC=C2)O',
'O=S(NS(=O)(C(F)(F)F)=O)(C(F)(F)F)=O',
'O=S(C1=CC=CC=C1C2=CN=NC(C3=NC=C(C4=CC=CC=C4S(=O)([O-])=O)C=C3)=N2)([O-])=O.[Na+].[Na+]',
'CCCC[N+](CCCC)(CCCC)CCCC.[O-]S(=O)(O)=O',
'O=P(OC1=CC=C2C(C=CC=C2)=C31)(O)OC4=C3C(C=CC=C5)=C5C=C4']
solvent_SMILES=['ClC(Cl)Cl',
                'ClCCCl',
                'ClCCl',
                'CCC(C)(O)C',
                'CC1=CC=CC=C1',
                'CC1=CC(C)=CC(C)=C1',
                'FC(F)(F)C1=CC=CC=C1',
                'FC1=C(F)C(F)=C(F)C(F)=C1F',
                'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
                'C[N+]([O-])=O',
                '[H]O[H]',
                'C1CCOC1',
                'O1CCOCC1']
cosolvent_SMILES=['FC(F)(F)C(O)C(F)(F)F',
                  r'F/C(F)=C(F)\F',
                  'FC(F)(F)C(C(F)(F)F)(O)C(F)(F)F',
                  'CC(C)(O)C',
                  'CC(O)C']
additives_SMILES=['O=S([O-])([O-])=O.[Na+].[Na+]',
                  'CC(C1=CC(C)=CC(C(C)(C)C)=C1O)(C)C',
                  '[H]O[H]','[O-][Si](=O)[O-].[O-][Si](=O)[O-].[Na+].[Al+3]',
                  'O=C1CC[C@@]2([H])[C@]3([H])CC=C4C[C@@H](OC(C)=O)CC[C@]4(C)[C@@]3([H])CC[C@]12C',
                  'CO','O1CCOCC1']

In [39]:
MCP_SMILES_dic={
    'parent Ph-MCP':r'C1(/C=C2CC\2)=CC=CC=C1', 
    '4-Br-Ph-MCP':r'BrC(C=C1)=CC=C1/C=C2CC\2', 
    '4-Cl-Ph-MCP':r'ClC(C=C1)=CC=C1/C=C2CC\2', 
    '4-F-Ph-MCP':r'FC(C=C1)=CC=C1/C=C2CC\2', 
    '4-nitro-Ph-MCP':r'O=[N+](C(C=C1)=CC=C1/C=C2CC\2)[O-]',
    '4-MeO-Ph-MCP':r'COC(C=C1)=CC=C1/C=C2CC\2',
    '4-Me2N-Ph-MCP':r'CN(C(C=C1)=CC=C1/C=C2CC\2)C',
    '4-iPr-Ph-MCP':r'CC(C)C(C=C1)=CC=C1/C=C2CC\2',
    '4-Ph-Ph-MCP':r'C1(/C=C2CC\2)=CC=C(C3=CC=CC=C3)C=C1',
    '3-CH3-Ph-MCP':r'CC1=CC(/C=C2CC\2)=CC=C1',
    '3-Br-Ph-MCP':r'BrC1=CC(/C=C2CC\2)=CC=C1',
#     '2-CH3-Ph-MCP':r'CC1=CC=CC=C1/C=C2CC\2',
    '3-iPr-Ph-MCP':r'CC(C)C1=CC(/C=C2CC\2)=CC=C1',
    '4-CF3-Ph-MCP':r'FC(C(C=C1)=CC=C1/C=C2CC\2)(F)F',
    '4-CH3-Ph-MCP':'CC(C=C1)=CC=C1/C=C2CC\2'}
solvent_SMILES_dic={
    'chloroform':'ClC(Cl)Cl',
                '1,2-dichloroethane':'ClCCCl',
                'dichloromethane':'ClCCl',
                'tert-amyl alcohol':'CCC(C)(O)C',
                'toluene':'CC1=CC=CC=C1',
                'mesitylene':'CC1=CC(C)=CC(C)=C1',
                'α,α,α-trifluorotoluene':'FC(F)(F)C1=CC=CC=C1',
                'hexafluorobenzene':'FC1=C(F)C(F)=C(F)C(F)=C1F',
                'perfluorohexane':'FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F',
                'nitromethane':'C[N+]([O-])=O',
                'water':'[H]O[H]',
                'THF':'C1CCOC1',
                '1,4-dioxane':'O1CCOCC1'}

Catalyst_SMILES_dic={
    'ptsa': 'CC1=CC=C(S(O)(=O)=O)C=C1',
    'benzenesulfonic acid': 'O=S(C1=CC=CC=C1)(O)=O',
                  'MsOH':'CS(=O)(O)=O',
                  'HCl':'[H]Cl',
                  'HBF4':'F[B-](F)(F)F.[H+]',
                  'camphorsulfonic acid':'CC1(C)[C@]2([H])CC[C@@]1(CS(O)(=O)=O)C(C2)=O',
                  'pentafluorophenol':'OC1=C(F)C(F)=C(F)C(F)=C1F',
    'TfOH':'O=S(O)(C(F)(F)F)=O',
                  'HBr':'[H]Br',
    'HPF6':'F[P-](F)(F)(F)(F)F.[H+]',
                  'TFA':'O=C(O)C(F)(F)F',
                  'H3PO4':'O=P(O)(O)O',
                  'HClO4':'O=Cl(=O)(O)=O',
                  'HI':'[H]I',
                  'diphenyl hydrogen phosphate':'O=P(OC1=CC=CC=C1)(OC2=CC=CC=C2)O',
                  'HNTf2':'O=S(NS(=O)(C(F)(F)F)=O)(C(F)(F)F)=O',
                  'Pyridinium p-toluene sulfonate':'O=S(C1=CC=CC=C1C2=CN=NC(C3=NC=C(C4=CC=CC=C4S(=O)([O-])=O)C=C3)=N2)([O-])=O.[Na+].[Na+]',
                  'tetrabutylammonium hydrogensulfate':'CCCC[N+](CCCC)(CCCC)CCCC.[O-]S(=O)(O)=O',
                  'chiral BINOL phosphoric acid':'O=P(OC1=CC=C2C(C=CC=C2)=C31)(O)OC4=C3C(C=CC=C5)=C5C=C4'}

cosolvent_SMILES_dic={'HFIP':'FC(F)(F)C(O)C(F)(F)F',
    'TFE':r'F/C(F)=C(F)\F',
                  'nonafluoro-tert-butyl alcohol':'FC(F)(F)C(C(F)(F)F)(O)C(F)(F)F',
                  'tert-butanol':'CC(C)(O)C',
                  'iso-propanol':'CC(O)C'}

In [41]:
# Cosolvent_column_sterimol

In [54]:
X2D_catalyst=pd.read_csv('df_catalyst_pca_mordred_sterimol.csv').drop('Name',axis=1).to_numpy()
X2D_cosolvent=pd.read_csv('df_cosolvent_pca_mordred_sterimol.csv').drop('Name',axis=1).to_numpy()
X2D_solvent=pd.read_csv('df_solvent_pca_mordred_sterimol.csv').drop('Name',axis=1).to_numpy()
X2D_MCP=pd.read_csv('df_MCPs_pca_mordred_sterimol_Hammett_Gibbs.csv').drop(['Name','SMILES'],axis=1).to_numpy()

In [55]:
dict_solvent={solvent_SMILES[i]:X2D_solvent[i] for i in range(len(solvent_SMILES))}
dict_solvent['none']=np.array([0,0])


dict_catalyst={Catalyst_SMILES[i]:X2D_catalyst[i] for i in range(len(Catalyst_SMILES))}
dict_catalyst['none']=np.array([0,0])


dict_cosolvent={cosolvent_SMILES[i]:X2D_cosolvent[i] for i in range(len(cosolvent_SMILES))}
dict_cosolvent['none']=np.array([0,0])

dict_MCP={MCP_SMILES[i]:X2D_MCP[i] for i in range(len(MCP_SMILES_dic))}
dict_MCP['none']=np.array([0,0])

In [56]:
solvent_names=list(solvent_SMILES_dic.keys())
catalyst_names=list(Catalyst_SMILES_dic.keys())
cosolvent_names=list(cosolvent_SMILES_dic.keys())
MCP_names=list(MCP_SMILES_dic.keys())

In [57]:
solvent_list=pd.read_csv('suggestions_data_derivatives1.csv')[['Solvent dim1','Solvent dim2']].to_numpy()
solvent_list_decimal=[[round(i[0],4),round(i[1],4)] for i in solvent_list]
dict_solvent_keylist=list(dict_solvent.keys())
dict_solvent_valuelist_decimal=[[round(i[0],4),round(i[1],4)] for i in np.array(list(dict_solvent.values()))]
solvent_column_sterimol=[dict_solvent_keylist[dict_solvent_valuelist_decimal.index(i)] for i in solvent_list_decimal]

Catalyst_list=pd.read_csv('suggestions_data_derivatives1.csv')[['Catalyst dim1','Catalyst dim2']].to_numpy()
Catalyst_list_decimal=[[round(i[0],5),round(i[1],5)] for i in Catalyst_list]
dict_Catalyst_keylist=list(dict_catalyst.keys())
dict_Catalyst_valuelist_decimal=[[round(i[0],5),round(i[1],5)] for i in np.array(list(dict_catalyst.values()))]
Catalyst_column_sterimol=[dict_Catalyst_keylist[dict_Catalyst_valuelist_decimal.index(i)] for i in Catalyst_list_decimal]

Cosolvent_list=pd.read_csv('suggestions_data_derivatives1.csv')[['Cosolvent dim1','Cosolvent dim2']].to_numpy()
Cosolvent_list_decimal=[[round(i[0],5),round(i[1],5)] for i in Cosolvent_list]
dict_Cosolvent_keylist=list(dict_cosolvent.keys())
dict_Cosolvent_valuelist_decimal=[[round(i[0],5),round(i[1],5)] for i in np.array(list(dict_cosolvent.values()))]
Cosolvent_column_sterimol=[dict_Cosolvent_keylist[dict_Cosolvent_valuelist_decimal.index(i)] for i in Cosolvent_list_decimal]

MCP_list=pd.read_csv('suggestions_data_derivatives1.csv')[['MCP dim1','MCP dim2']].to_numpy()
MCP_list_decimal=[[round(i[0],5),round(i[1],5)] for i in MCP_list]
dict_MCP_keylist=list(dict_MCP.keys())
dict_MCP_valuelist_decimal=[[round(i[0],5),round(i[1],5)] for i in np.array(list(dict_MCP.values()))]
MCP_column_sterimol=[dict_MCP_keylist[dict_MCP_valuelist_decimal.index(i)] for i in MCP_list_decimal]

In [59]:
dict_solvent_names=[solvent_names[dict_solvent_keylist.index(i)] for i in solvent_column_sterimol]
solvent_names_column_sterimol=pd.DataFrame(dict_solvent_names).to_csv('solvent_names_column_derivatives.csv')

In [60]:
dict_Catalyst_names=[catalyst_names[dict_Catalyst_keylist.index(i)] for i in Catalyst_column_sterimol]
Catalyst_names_column_sterimol=pd.DataFrame(dict_Catalyst_names).to_csv('Catalyst_names_column_derivatives.csv')

In [61]:
dict_Cosolvent_names=[cosolvent_names[dict_Cosolvent_keylist.index(i)] for i in Cosolvent_column_sterimol]
Cosolvent_names_column_sterimol=pd.DataFrame(dict_Cosolvent_names).to_csv('Cosolvent_names_column_derivatives.csv')

In [62]:
dict_MCP_names=[MCP_names[dict_MCP_keylist.index(i)] for i in MCP_column_sterimol]
MCP_names_column_sterimol=pd.DataFrame(dict_MCP_names).to_csv('MCP_names_column_derivatives.csv')

-----------------------------------------------------------

In [72]:
# Evaluate the batch to get the corresponding target values

raw_y = exp_1(raw_X)
raw_y

[[-10.50122051  -2.15758673 -11.14697706  -9.27334807  14.36486742
   -1.75190403 -13.91750442  -7.75329141  21.          10.
    4.        ]
 [-10.50122051  -2.15758673 -11.14697706  -9.27334807  14.36486742
   -1.75190403 -13.91750442  -7.75329141  21.          10.
    2.        ]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


array([[2.],
       [1.]])

In [76]:
# implement sequential loop (this is equivalent to the run_experiment method)

for i in range(11, 16):
    raw_X = boc.get_next_X()
    raw_y = exp_1(raw_X)
    boc.update_experiment(raw_X, raw_y)
    print(f'epoch: {i}\n', f'X: {raw_X}\n', f'y: {raw_y}\n')

[[  1.08309219   0.60956586 -11.14697706  -9.27334807  14.36486742
   -1.75190403  35.68073466   5.57456039  21.          10.
    2.        ]
 [  1.08309219   0.60956586 -11.14697706  -9.27334807  14.36486742
   -1.75190403  35.68073466   5.57456039  21.          10.
    8.        ]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
epoch: 11
 X: [[  1.08309219   0.60956586 -11.14697706  -9.27334807  14.36486742
   -1.75190403  35.68073466   5.57456039  21.          10.
    2.        ]
 [  1.08309219   0.60956586 -11.14697706  -9.27334807  14.36486742
   -1.75190403  35.68073466   5.57456039  21.          10.
    8.        ]]
 y: [[30.]
 [28.]]

Non 200 response. Response object returned
<Response [500]>
<class 'bytes'>


AttributeError: 'bytes' object has no attribute 'astype'

In [ ]:
plot_progress(boc)

### Check the state of the experiment

In [19]:
exp_details = boc.get_experiment_details()


In [20]:
exp_details

{'message': 'Either token/experiment does not exist or the token does not have access to the given token'}

## Kill the experiment

In [140]:
m = boc.kill_experiment()
print(m)

Experiment is being deleted


In [141]:
# Check it's actually dead
m = boc.readiness_check()
print(m)

Experiment is not ready. Status: Deleting


In [13]:
from time import sleep
l = [x[0] for x in boc.list_experiments(status=True)]
for x in l:
    boc.experiment_id = x
    boc.kill_experiment()
    sleep(5)
    print(x,' is deleted')

Multiple Ids retrieved.
00000000-0000-0000-0006-0047a3b6996d  is deleted
00000000-0000-0000-0006-0047b3fab252  is deleted
00000000-0000-0000-0006-00495b9b80a7  is deleted
00000000-0000-0000-0006-004972b43c2f  is deleted
00000000-0000-0000-0006-004989dddf3d  is deleted
00000000-0000-0000-0006-0049a0dfb1ea  is deleted
00000000-0000-0000-0006-0049b7e028b2  is deleted
00000000-0000-0000-0006-0049cee679f0  is deleted
00000000-0000-0000-0006-0049e5e5798d  is deleted
00000000-0000-0000-0006-004ac9b5e11d  is deleted
00000000-0000-0000-0006-004ae4b01028  is deleted
00000000-0000-0000-0006-004b29cb0ea0  is deleted


## Archive & Restore experiment

Eventually the user may not wish to delete an experiemtn but archive it instead. Freeing up their resources but keeping the information submitted to an experiment, it can then be later restored if desired and the experiment resumed.

Frist let us create an experiment and submit some values to it.

In [144]:
boc = BOC(user_token=TOKEN, ip=IP, port=PORT)
boc.generate_experiment(config, wait=True)
boc.run_experiment(objective_f=exp_1, epochs=10)
boc.get_next_X()

New experiment setup with id: 00000000-0000-0000-0005-fdb6e7f71c73
[[  1.08309219   0.60956586 -11.14697706  -9.27334807  -8.24574092
   28.16949142   0.           0.          21.          10.
    2.        ]
 [  1.08309219   0.60956586 -11.14697706  -9.27334807 -21.7728893
   -7.41501346   0.           0.          21.          10.
    8.        ]]
epoch: 0
 X: [[  1.08309219   0.60956586 -11.14697706  -9.27334807  -8.24574092
   28.16949142   0.           0.          21.          10.
    2.        ]
 [  1.08309219   0.60956586 -11.14697706  -9.27334807 -21.7728893
   -7.41501346   0.           0.          21.          10.
    8.        ]]
 y: [[21.]
 [ 0.]]

[[  1.08309219   0.60956586 -11.14697706  -9.27334807  -8.24574092
   28.16949142   0.           0.          21.          10.
    4.        ]]
epoch: 1
 X: [[  1.08309219   0.60956586 -11.14697706  -9.27334807  -8.24574092
   28.16949142   0.           0.          21.          10.
    4.        ]]
 y: [[23.]]

[[  1.08309219   0.6

array([[ 1.08309219,  0.60956586,  7.7513066 , 20.24748829, 14.36486742,
        -1.75190403,  0.        ,  0.        , 21.        , 10.        ,
         2.        ]])

In [ ]:
boc.list_experiments(status=True)

Now archive the experiment and wait for the archiving process to complete

In [25]:
boc.archive_experiment()

'Can not archive an experiment that is not in a ready state, status: Deleting'

In [65]:
from time import sleep
l = [x[1] for x in boc.list_experiments(status=True)]
stat = (l[0]=='Archived')
#If only 1 experiment, reference 5-8 lines
while not stat:
    sleep(5)
    l = [x[1]=='Ready' for x in boc.list_experiments(status=True)]
    stat = (l[0]=='Archived')
boc.list_experiments(status=True)

Multiple Ids retrieved.
Multiple Ids retrieved.


[['00000000-0000-0000-0005-fc63ddf7ebbf', 'Archived'],
 ['00000000-0000-0000-0005-fc640cdec244', 'Ready'],
 ['00000000-0000-0000-0005-fc641fb59a68', 'ArchivingInProgress']]

With it archived we can still check the details of the experiment

In [9]:
boc.get_experiment_details()

C:\Users\cc2819\anaconda3\envs\boc\lib\site-packages\boc\boc.py:318: UserWarning: Memory at 8888.83%, please consider contacting your admin to upgrade the resources
  warnings.warn(resource_warning)


{'cachedValuesToReturn': [],
 'config': {'acq_kwargs': {'normalise_x': True,
   'normalise_y': True,
   'otypes': ['max']},
  'acq_type': 'Thompson',
  'bo_kwargs': {'init_obs': 1, 'log_level': 2, 'seed': 2},
  'dict_opt_kwargs': {'continuous': {'dim_types': ['continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous'],
    'domain': [[1.186137917,
      0.634095202,
      -11.21819749,
      -9.225527544,
      -9.22552754,
      -9.22552754,
      0.1,
      0.0,
      0.0,
      21.0,
      0.5,
      2.0],
     [1.186137917,
      0.634095202,
      -11.21819749,
      -9.225527544,
      -9.22552754,
      -9.22552754,
      0.1,
      0.0,
      0.0,
      21.0,
      1.0,
      2.0],
     [1.186137917,
      0.634095202,
      -11.21819749,
      -9.225527544,
      -9.22552754,
      -9.22552754,
      0.1,
      0.0,


At a later date we can then restore it

In [70]:
boc.restore_experiment()

Begginning Restoration process


In [71]:
from time import sleep
stat = all([x[1]=='Ready' for x in boc.list_experiments(status=True)])
# while not stat:
#     sleep(5)
#     stat = all([x[1]=='Ready' for x in boc.list_experiments(status=True)])
boc.list_experiments(status=True)

Multiple Ids retrieved.
Multiple Ids retrieved.


[['00000000-0000-0000-0005-fc63ddf7ebbf', 'Archived'],
 ['00000000-0000-0000-0005-fc640cdec244', 'Ready'],
 ['00000000-0000-0000-0005-fc641fb59a68', 'Restoring']]

Now that the experiment is restored the `get_next_X` values should be the same/similar to those that we given just before we archived the experiment

In [42]:
boc.get_next_X()

array([[ 0.235925  , -0.69554882]])

In [8]:
boc.experiment_id = '00000000-0000-0000-0006-0026067429ba'

In [156]:
boc.kill_experiment()

'Experiment is being deleted'

In [7]:
boc.list_experiments(status=True)

Multiple Ids retrieved.


[['00000000-0000-0000-0006-002217506420', 'Ready'],
 ['00000000-0000-0000-0006-00236b0df1bc', 'Ready'],
 ['00000000-0000-0000-0006-0023c4bd979a', 'Ready'],
 ['00000000-0000-0000-0006-00258296f6e9', 'Ready'],
 ['00000000-0000-0000-0006-0026067429ba', 'Ready']]

If archived, unable to be deleted; 